<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Testing_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorflow as tf

In [ ]:
a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([[5, 6], [7, 8]])
print(a.reshape(b.shape))
print(torch.reshape(a, b.shape))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [ ]:
def gather_old(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    axis = abs(len(params.shape) + axis) if axis < 0 else axis
    print(f"AXIS: {axis}")
    batch_dims %= len(params.shape)
    result = []

    def expand_p_i(params, indices, axis = axis):
        ind_dim_helper_table = [1 for dim in list(indices.shape)]
        ind_singleton_dims = torch.Size(ind_dim_helper_table)
        param_singleton_dims_table = [1 for dim in list(params.shape)]
        param_singleton_dims = torch.Size(param_singleton_dims_table)
        print("INDICES SHAPE: ", indices.shape)
        print("PARAMS SHAPE: ", params.shape)
        print(f"SINGLETON_DIMS: {param_singleton_dims}, {ind_singleton_dims}")
        print("RESHAPE SHAPE: ", param_singleton_dims[:axis]
                                     + indices.shape[batch_dims:]
                                     + param_singleton_dims[axis + 1:], "\n",(params.shape[:axis]
                                   + ind_singleton_dims[batch_dims: - 1] + max([ind_singleton_dims[-1:], params.shape[axis:axis + 1]])
                                   + params.shape[axis + 1:]))
        print(param_singleton_dims[:axis],
                                      indices.shape[batch_dims:],
                                      param_singleton_dims[axis + 1:], batch_dims)
        print("SIZE TUPLES: ", indices.shape[batch_dims:], params.shape[axis:axis + 1], params.shape, params)
        params_insert_shape = ((torch.tensor(indices.shape[batch_dims:]) == params.shape[axis % len(params)]) * params.shape[axis % len(params)]).long() + (torch.tensor(indices.shape[batch_dims:]) != params.shape[axis % len(params)]).long()
        params_insert_shape = [dim for dim in params_insert_shape]
        print(torch.Size(params_insert_shape) != torch.Size([0]))
        params_insert_shape = torch.Size(params_insert_shape) if torch.Size(params_insert_shape) != torch.Size([0]) else indices.shape[batch_dims:]
        print(f"\nParams insert shape: {params_insert_shape}", " params reshape shape: ", (params.shape[:axis]
                                   + params_insert_shape
                                   + params.shape[axis + 1:]), "Params shape: ", params.shape)
        indices_ex = indices if (params.dim() <= 1) else \
                    indices.reshape(param_singleton_dims[:axis]
                                     + indices.shape[batch_dims:]
                                     + param_singleton_dims[axis + 1:])
        params_shape = list(params.shape[:axis]
                                   + params_insert_shape
                                   + params.shape[axis + 1:])
        if params.shape[axis % len(params)] not in params_shape:
          params_shape[-1] = params.shape[axis]
        params_shape = torch.Size(params_shape)
        new_axis = (axis + len(params_insert_shape) - 1) % len(params_shape)
        print(f"AXIS = {axis}, NEW_AXIS = {new_axis}")
        params_ex = params if indices_ex.dim() == params.dim() else params.reshape(params_shape)

        print("FINAL SHAPE: ", params.shape[:axis]
                                         + indices.shape[batch_dims:]
                                         + params.shape[axis + 1:], params_ex.shape, indices_ex.shape)

        params_ex_mask = torch.tensor(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]) * (torch.tensor(params_ex.shape) == 1)
        print(f"PARAMS_EX_MASK = {params_ex.shape}, {params_ex_mask}, {torch.tensor(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:])}")
        print(params_ex_mask, (torch.tensor(params_ex.shape) == 1))
        print("CONDITIONS FOR MASKS- PARAMS: ", (params_ex_mask != (torch.tensor(params_ex.shape) == 1).flatten()))
        if any((params_ex_mask != (torch.tensor(params_ex.shape) == 1).flatten())):
            params_ex = params_ex.expand(params.shape[:axis]
                                         + indices.shape[batch_dims:]
                                         + params.shape[axis + 1:])
        indices_ex_mask = torch.tensor(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]) * (torch.tensor(indices_ex.shape) == 1)
        print("CONDITIONS FOR MASKS- INDICES: ", (indices_ex_mask != (torch.tensor(indices_ex.shape) == 1).flatten()))
        if any((indices_ex_mask != (torch.tensor(indices_ex.shape) == 1).flatten())):
            indices_ex = indices_ex.expand(params.shape[:axis]
                                           + indices.shape[batch_dims:]
                                           + params.shape[axis + 1:])
        print("EXPANDED SHAPES: ", params_ex.shape, indices_ex.shape)
        return params_ex, indices_ex, new_axis
    if batch_dims == 0:
        dim_diff = params.dim() - indices.dim()
        print(f"DIM_DIFF= {dim_diff}")
        if dim_diff != 0:
            params_expanded, indices_expanded, new_axis = expand_p_i(params, indices)
            result = torch.gather(
                                    params_expanded, new_axis,
                                    indices_expanded.long(),
                                    sparse_grad=False, out=out
                                 )
            result = result.to(dtype = params.dtype)
            return result
        else:
            params_expanded, indices_expanded, new_axis = expand_p_i(params, indices)
            print(f"PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = {params_expanded}, {indices_expanded}")
            result = torch.gather(
                params_expanded, new_axis, indices_expanded.long(), sparse_grad=False, out=out)
            result = result.to(dtype = params.dtype)

    else:
        params_slices = torch.unbind(params, axis=0) if params.dim() > 1 else params
        indices_slices = torch.unbind(indices, axis=0) if indices.dim() > 1 else indices
        print(f"SLICES: {params_slices}, {indices_slices}")
        for b in range(batch_dims):
            if b == 0:
                zip_list = [(p, i) for
                            p, i in
                            zip(params_slices, indices_slices)
                            ]
            else:
                zip_list = [
                           (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list]
                           for p, i in z
                            ]
        for z in zip_list:
            p, i = z
            dim_diff = p.dim() - i.dim()
            p_expanded = p
            i_expanded = i
            if dim_diff != 0:
              p_expanded, i_expanded, new_axis = expand_p_i(p, i)
            print("p, i shapes: ", p_expanded.shape, i_expanded.shape)
            r = torch.gather(
                             p_expanded, (new_axis - batch_dims),
                             i_expanded.long(),
                             sparse_grad=False, out=None
                            )

            result.append(r)
        result = torch.stack(result)
        result = result.reshape(params.shape[:axis] + max(indices.shape[batch_dims:], torch.Size([1])) + params.shape[axis + 1:])
        result = result.to(dtype = params.dtype)
        print("FINAL RES SHAPE: ", result.shape, params.shape[:axis], indices.shape[batch_dims:], params.shape[axis + 1:])


    return result

In [ ]:
def gather_test(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    axis = abs(len(params.shape) + axis) if axis < 0 else axis
#    print(f"AXIS: {axis}")
    batch_dims %= len(params.shape)
    result = []

    def expand_p_i(params, indices, axis = axis, batch_dims = batch_dims):
        num_inserted_dims = len(indices.shape[batch_dims:])
        new_axis = axis
#        print(f"NEW_AXIS: {new_axis}")

        indices_ex = torch.clone(indices[batch_dims:])
        dim_diff = abs(params.dim() - indices[batch_dims:].dim())
        stack_dims1 = params.shape[:axis]
        stack_dims2 = params.shape[axis + 1:]
#        print(f"STACK_DIMS_1_2: {stack_dims1}, {stack_dims2}, {params.dim()}")
        for dim in range(params.dim()):
#          print(f"DIM = {dim}, {params.dim()}")
          if dim < axis:

            to_stack = [indices_ex for i in range(params.shape[dim])]
            indices_ex = torch.stack(to_stack, dim)
          if dim == axis:
            pass
          if dim > axis:
#            print(f"range(params.shape[dim]) = {range(params.shape[dim])}")
            to_stack = [indices_ex for i in range(params.shape[dim])]
            indices_ex = torch.stack(to_stack, dim + indices.dim() - 1)
        indices_ex = indices_ex.reshape((stack_dims1 + torch.Size([-1]) + stack_dims2))
#        print(f"INDICES SHAPE: {(stack_dims1, -1, stack_dims2)}, {indices_ex.shape}")
        return indices_ex, new_axis
    final_shape = params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]

#   print(f"FINAL SHAPE: {final_shape}")
    if batch_dims == 0:
        dim_diff = abs(params.dim() - indices.dim())
        if dim_diff != 0:
            indices_expanded, new_axis = expand_p_i(params, indices)
#            print(f"INDICES EXPANDED: {indices_expanded.shape}")

            result = torch.gather(
                                    params, new_axis,
                                    indices_expanded.long(),
                                    sparse_grad=False, out=out
                                 ).reshape(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:])
            result = result.to(dtype = params.dtype)
            return result
        else:
            indices_expanded, new_axis = expand_p_i(params, indices)
            result = torch.gather(
                params, new_axis, indices_expanded.long(), sparse_grad=False, out=out)
            result = result.to(dtype = params.dtype)

    else:
        indices_ex, new_axis = expand_p_i(params, indices)
        params_slices = torch.unbind(params, axis=0) if params.dim() > 1 else params
        indices_slices = torch.unbind(indices_ex, axis=0) if indices.dim() > 1 else indices
        for b in range(batch_dims):
            if b == 0:
                zip_list = [(p, i) for
                            p, i in
                            zip(params_slices, indices_slices)
                            ]
            else:
                zip_list = [
                           (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list]
                           for p, i in z
                            ]
        for z in zip_list:
            p, i = z

            r = torch.gather(
                             p, (new_axis - batch_dims),
                             i.long(),
                             sparse_grad=False, out=None
                            )

            result.append(r)
        result = torch.stack(result)
        result = result.reshape(params.shape[:axis] + max(indices.shape[batch_dims:], torch.Size([1])) + params.shape[axis + 1:])
        result = result.to(dtype = params.dtype)


    return result

In [2]:
def gather(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    axis = abs(len(params.shape) + axis) if axis < 0 else axis
#    print(f"AXIS: {axis}")
    batch_dims %= len(params.shape)
    result = []

    def expand_p_i(params, indices, axis = axis, batch_dims = batch_dims):
        axis %= len(params.shape)
        new_axis = axis
#        print(f"NEW_AXIS: {new_axis}")
#        print(f"indices.shape = {indices.shape}")
        indices_ex = torch.clone(indices).detach()
        dim_diff = abs(params.dim() - (indices_ex.dim() - batch_dims))
        stack_dims1 = params.shape[:axis]
        stack_dims2 = params.shape[axis + 1:]
#        print(f"params.shape = {params.shape}")
#        print(f"STACK_DIMS_1_2: {stack_dims1}, {stack_dims2}, {params.dim()}")
        indices_ex = indices_ex.reshape((torch.Size([1 for dim in stack_dims1]) + torch.Size([-1]) + torch.Size([1 for dim in stack_dims2])))
        indices_ex = indices_ex.expand((stack_dims1 + torch.Size([-1]) + stack_dims2)).reshape((stack_dims1 + torch.Size([-1]) + stack_dims2))
#        print(f"INDICES SHAPE: {(stack_dims1 + torch.Size([-1]) + stack_dims2)}, {indices_ex.shape}")
        return indices_ex, new_axis
    final_shape = params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]

#    print(f"FINAL SHAPE: {final_shape}, indices.shape[batch_dims:] = {indices.shape[batch_dims:]}")
    if batch_dims == 0:
        dim_diff = abs(params.dim() - (indices.dim() - batch_dims))
        if dim_diff != 0:
            indices_expanded, new_axis = expand_p_i(params, indices)
#            print(f"INDICES EXPANDED: {indices_expanded.shape}")

            result = torch.gather(
                                    params, new_axis,
                                    indices_expanded.long(),
                                    sparse_grad=False, out=out
                                 ).reshape(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:])
            result = result.to(dtype = params.dtype)
            return result
        else:
            indices_expanded, new_axis = expand_p_i(params, indices)
            result = torch.gather(
                params, new_axis, indices_expanded.long(), sparse_grad=False, out=out).reshape(final_shape)
            result = result.to(dtype = params.dtype)

    else:
#        indices_ex, new_axis = expand_p_i(params, indices)
        indices_ex = indices
        new_axis = axis
        params_slices = torch.unbind(params, axis=0) if params.shape[0] > 0 else params
        indices_slices = torch.unbind(indices_ex, axis=0) if indices.shape[0] > 0 else indices_ex
#        print(f"INDICES SLICE: {indices_slices}, PARAMS SLICES: {params_slices}")
#        print(f"indices_slices[0].shape = {indices_slices[0].shape}")
        for b in range(batch_dims):
#            print(f"{b} out of {range(batch_dims)}")
            if b == 0:
                zip_list = [(p, i) for
                            p, i in
                            zip(params_slices, indices_slices)
                            ]
            else:
                zip_list = [
                           (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list]
                           for p, i in z
                            ]
#        print(f"ZIP LIST LEN: {len(zip_list)}")
#        print(f"ZIP LIST: {zip_list}")
        for z in zip_list:
            p, i = z
            i_ex, new_axis = expand_p_i(p, i, axis = axis - batch_dims)
#            i_ex = i
#            print(f"I SHAPE: {i_ex.shape}, P SHAPE: {p.shape}, NEW_AXIS - BATCH_DIMS = {new_axis - batch_dims}")
            r = torch.gather(
                             p, (new_axis),
                             i_ex.long(),
                             sparse_grad=False, out=None
                            )
#            print(f"R SHAPE: {r.shape}")
            result.append(r)
        result = torch.stack(result)
#        print(f"FINAL RESULT SHAPE: {result.shape}")
        result = result.reshape(params.shape[:axis] + max(indices.shape[batch_dims:], torch.Size([1])) + params.shape[axis + 1:])
        result = result.to(dtype = params.dtype)


    return result

In [3]:
params = torch.tensor([-1.], dtype=torch.float16)

indices = torch.tensor([[0, 0, 0],
                   [0, 0, 0]], dtype=torch.int32)

paramstf = tf.constant([-1.], dtype=tf.float16)

indicestf = tf.constant([[0, 0, 0],
                   [0, 0, 0]], dtype=tf.int32)

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
torch_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor)
tf_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[-1. -1. -1.]
 [-1. -1. -1.]], shape=(2, 3), dtype=float16)


torch:  tensor([[-1., -1., -1.],
        [-1., -1., -1.]], dtype=torch.float16)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]], shape=(2, 3), dtype=bool)


In [ ]:
params = torch.tensor([[-1.],
                 [ 1.],
                 [ 1.]])

indices = torch.tensor([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

paramstf = tf.constant([[-1.],
                 [ 1.],
                 [ 1.]])

indicestf = tf.constant([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

axis = 1
batch_dims = 1

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
torch_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor)
tf_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print()
print(torch_tensor)
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[-1.]
  [-1.]]

 [[ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]]], shape=(3, 2, 1), dtype=float32)


FINAL SHAPE: torch.Size([3, 2, 1]), indices.shape[batch_dims:] = torch.Size([2, 1])
INDICES SLICE: (tensor([[0],
        [0]]), tensor([[0],
        [0]]), tensor([[0],
        [0]])), PARAMS SLICES: (tensor([-1.]), tensor([1.]), tensor([1.]))
indices_slices[0].shape = torch.Size([2, 1])
0 out of range(0, 1)
ZIP LIST LEN: 3
ZIP LIST: [(tensor([-1.]), tensor([[0],
        [0]])), (tensor([1.]), tensor([[0],
        [0]])), (tensor([1.]), tensor([[0],
        [0]]))]
indices.shape = torch.Size([2, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
I SHAPE: torch.Size([2]), P SHAPE: torch.Size([1]), NEW_AXIS - BATCH_DIMS = -1
R SHAPE: torch.Size([2])
indices.shape = torch.Size([2, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.S

In [ ]:
params = torch.tensor([[[1, 1]]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[[1, 1]]])

indicestf = tf.constant([[0]])



axis = 1
batch_dims = 1
print(tf.gather(tf.constant([[1, 1]]), indices = torch.tensor([0]), axis = 0))
print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))

print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf.Tensor([[1 1]], shape=(1, 2), dtype=int32)
torch.Size([1, 1, 2])
tensor([[0]])
3 2
tf:  tf.Tensor([[[1 1]]], shape=(1, 1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 1, 2]), indices.shape[batch_dims:] = torch.Size([1])
INDICES SLICE: (tensor([0]),), PARAMS SLICES: (tensor([[1, 1]]),)
indices_slices[0].shape = torch.Size([1])
0 out of range(0, 1)
ZIP LIST LEN: 1
ZIP LIST: [(tensor([[1, 1]]), tensor([0]))]
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
I SHAPE: torch.Size([1, 2]), P SHAPE: torch.Size([1, 2]), NEW_AXIS - BATCH_DIMS = -1
R SHAPE: torch.Size([1, 2])
FINAL RESULT SHAPE: torch.Size([1, 1, 2])
torch:  tensor([[[1, 1]]])
FINAL SHAPE: torch.Size([1, 1, 2]), indices.shape[batch_dims:] = torch.Size([1])
INDICES SLICE: (tensor([0]),), PARAMS SLICES: (tensor([[1, 1]]),)
indices_slices[0].shape = torch.Size([1])
0 out of range(0, 1)
ZIP LIST LEN: 1
ZIP LI

In [ ]:
params = torch.tensor([[0], [1]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[0], [1]])

indicestf = tf.constant([[0]])



axis = 1
batch_dims = 0
print(torch.gather(params, axis, indices))
print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(torch.gather(params, axis, indices))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[0]])
torch.Size([2, 1])
tensor([[0, 0]])
tensor([[0]])
2 2
tf:  tf.Tensor(
[[[0]]

 [[1]]], shape=(2, 1, 1), dtype=int32)


FINAL SHAPE: torch.Size([2, 1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([2]), torch.Size([]), 2
INDICES SHAPE: torch.Size([2, -1]), torch.Size([2, 1])
torch:  tensor([[[0]],

        [[1]]])
FINAL SHAPE: torch.Size([2, 1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([2]), torch.Size([]), 2
INDICES SHAPE: torch.Size([2, -1]), torch.Size([2, 1])
torch_to_tf:  tf.Tensor(
[[[0]]

 [[1]]], shape=(2, 1, 1), dtype=int32)

tf.Tensor(
[[[ True]]

 [[ True]]], shape=(2, 1, 1), dtype=bool)


In [ ]:
params = torch.tensor([[0], [0], [1]])

indices = torch.tensor([0])

paramstf = tf.constant([[0], [0], [1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 1

print(params.dim(), indices.dim())
print(paramstf.shape[:axis] + indicestf.shape[batch_dims:] + paramstf.shape[axis + 1:])
print(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:])

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

2 1
(1,)
torch.Size([1])
tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([])
INDICES SLICE: (tensor(0),), PARAMS SLICES: (tensor([0]), tensor([0]), tensor([1]))
indices_slices[0].shape = torch.Size([])
0 out of range(0, 1)
ZIP LIST LEN: 1
ZIP LIST: [(tensor([0]), tensor(0))]
indices.shape = torch.Size([])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
I SHAPE: torch.Size([1]), P SHAPE: torch.Size([1]), NEW_AXIS - BATCH_DIMS = -1
R SHAPE: torch.Size([1])
FINAL RESULT SHAPE: torch.Size([1, 1])
torch:  tensor([[0]])
FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([])
INDICES SLICE: (tensor(0),), PARAMS SLICES: (tensor([0]), tensor([0]), tensor([1]))
indices_slices[0].shape = torch.Size([])
0 out of range(0, 1)
ZIP LIST LEN: 1
ZIP LIST: [(tensor([0]), tensor(0))]
indices.shape = torch.Size([])
params.shape = torch.

In [ ]:
params = torch.tensor([[     367001,           0,      -51929,           0],
                 [      43690,        -135,  1952257859,           0],
                 [        272,           0,       69127,          93],
                 [   83886426,      -53715, -1073551173,          15],
                 [-1127219200,       71424,  -654416436,  -369361152]])

indices = torch.tensor([[[0, 0, 1, 3, 1, 0],
                  [3, 0, 0, 0, 0, 0],
                  [0, 0, 1, 2, 2, 0],
                  [3, 1, 3, 1, 1, 0],
                  [1, 2, 0, 0, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 1],
                  [3, 1, 3, 1, 1, 0]],

                 [[1, 1, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 0, 1, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 1, 0, 3, 0],
                  [0, 3, 0, 0, 0, 0]]])

paramstf = tf.constant([[     367001,           0,      -51929,           0],
                 [      43690,        -135,  1952257859,           0],
                 [        272,           0,       69127,          93],
                 [   83886426,      -53715, -1073551173,          15],
                 [-1127219200,       71424,  -654416436,  -369361152]])

indicestf = tf.constant([[[0, 0, 1, 3, 1, 0],
                  [3, 0, 0, 0, 0, 0],
                  [0, 0, 1, 2, 2, 0],
                  [3, 1, 3, 1, 1, 0],
                  [1, 2, 0, 0, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 1],
                  [3, 1, 3, 1, 1, 0]],

                 [[1, 1, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 0, 1, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 1, 0, 3, 0],
                  [0, 3, 0, 0, 0, 0]]])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
torch_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor)
tf_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print()
print(torch_tensor)
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[[     367001           0      -51929           0]
   [     367001           0      -51929           0]
   [      43690        -135  1952257859           0]
   [   83886426      -53715 -1073551173          15]
   [      43690        -135  1952257859           0]
   [     367001           0      -51929           0]]

  [[   83886426      -53715 -1073551173          15]
   [     367001           0      -51929           0]
   [     367001           0      -51929           0]
   [     367001           0      -51929           0]
   [     367001           0      -51929           0]
   [     367001           0      -51929           0]]

  [[     367001           0      -51929           0]
   [     367001           0      -51929           0]
   [      43690        -135  1952257859           0]
   [        272           0       69127          93]
   [        272           0       69127          93]
   [     367001           0      -51929           0]]

  [[   83886426      -53

In [ ]:
params = torch.tensor([[     367001,           0,      -51929,           0],
                 [      43690,        -135,  1952257859,           0],
                 [        272,           0,       69127,          93],
                 [   83886426,      -53715, -1073551173,          15],
                 [-1127219200,       71424,  -654416436,  -369361152]])

indices = torch.tensor([[[0, 0, 1, 3, 1, 0],
                  [3, 0, 0, 0, 0, 0],
                  [0, 0, 1, 2, 2, 0],
                  [3, 1, 3, 1, 1, 0],
                  [1, 2, 0, 0, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 1],
                  [3, 1, 3, 1, 1, 0]],

                 [[1, 1, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 0, 1, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 1, 0, 3, 0],
                  [0, 3, 0, 0, 0, 0]]])
paramstf = tf.constant([[     367001,           0,      -51929,           0],
                 [      43690,        -135,  1952257859,           0],
                 [        272,           0,       69127,          93],
                 [   83886426,      -53715, -1073551173,          15],
                 [-1127219200,       71424,  -654416436,  -369361152]])

indicestf = tf.constant([[[0, 0, 1, 3, 1, 0],
                  [3, 0, 0, 0, 0, 0],
                  [0, 0, 1, 2, 2, 0],
                  [3, 1, 3, 1, 1, 0],
                  [1, 2, 0, 0, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 1],
                  [3, 1, 3, 1, 1, 0]],

                 [[1, 1, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 0, 1, 3, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 0, 1, 0, 3, 0],
                  [0, 3, 0, 0, 0, 0]]])

print(indices)
#indices = indices.flatten()
print(indices.shape)
indices = torch.tensor(torch.stack([indices, indices, indices, indices], -1)).reshape((-1, 4))
print(indices.shape)
res = torch.gather(params, 0, indices)
res = res.flatten()
res = res.reshape((2, 8, 6, 4))
res_to_tf = tf.convert_to_tensor(res)
res_to_tf = tf.cast(res_to_tf, dtype = "int32")
print(res.long())
tf_res = tf.gather(paramstf, indicestf, axis = 0)
print(tf_res == res_to_tf)

tensor([[[0, 0, 1, 3, 1, 0],
         [3, 0, 0, 0, 0, 0],
         [0, 0, 1, 2, 2, 0],
         [3, 1, 3, 1, 1, 0],
         [1, 2, 0, 0, 3, 1],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1],
         [3, 1, 3, 1, 1, 0]],

        [[1, 1, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 2, 1],
         [0, 0, 0, 0, 0, 0],
         [2, 0, 0, 1, 3, 1],
         [0, 0, 0, 0, 0, 0],
         [2, 0, 1, 0, 3, 0],
         [0, 3, 0, 0, 0, 0]]])
torch.Size([2, 8, 6])
torch.Size([96, 4])
tensor([[[[     367001,           0,      -51929,           0],
          [     367001,           0,      -51929,           0],
          [      43690,        -135,  1952257859,           0],
          [   83886426,      -53715, -1073551173,          15],
          [      43690,        -135,  1952257859,           0],
          [     367001,           0,      -51929,           0]],

         [[   83886426,      -53715, -1073551173,          15],
          [     367001,           0,    

<ipython-input-267-e3a8d87d072d>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices = torch.tensor(torch.stack([indices, indices, indices, indices], -1)).reshape((-1, 4))


In [ ]:
params = torch.tensor([-54,   0,   1,   7, -60,   1])

indices = torch.tensor([[1, 0, 2, 2, 1, 1]])

paramstf = tf.constant([-54,   0,   1,   7, -60,   1])

indicestf = tf.constant([[1, 0, 2, 2, 1, 1]])

axis = 0
batch_dims = 0
print(torch.gather(params.reshape((1, 6)), axis - 1, indices))
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[  0, -54,   1,   1,   0,   0]])
tf:  tf.Tensor([[  0 -54   1   1   0   0]], shape=(1, 6), dtype=int32)


FINAL SHAPE: torch.Size([1, 6]), indices.shape[batch_dims:] = torch.Size([1, 6])
indices.shape = torch.Size([1, 6])
params.shape = torch.Size([6])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([6])
torch:  tensor([[  0, -54,   1,   1,   0,   0]])
FINAL SHAPE: torch.Size([1, 6]), indices.shape[batch_dims:] = torch.Size([1, 6])
indices.shape = torch.Size([1, 6])
params.shape = torch.Size([6])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([6])
torch_to_tf:  tf.Tensor([[  0 -54   1   1   0   0]], shape=(1, 6), dtype=int32)

tf.Tensor([[ True  True  True  True  True  True]], shape=(1, 6), dtype=bool)


In [ ]:
params = torch.tensor([0, 1])

indices = torch.tensor([[0, 0]])

paramstf = tf.constant([0, 1])

indicestf = tf.constant([[0, 0]])

axis = 0
batch_dims = 0
#print(torch.tensor(indices.shape[batch_dims:])!= params.shape[axis])
#params_insert_shape = ((torch.tensor(indices.shape[batch_dims:]) == params.shape[axis:axis + 1][0]) * params.shape[axis : axis + 1][0]).long() + (torch.tensor(indices.shape[batch_dims:]) != params.shape[axis : axis + 1][0]).long()
#params_insert_shape = torch.Size([dim for dim in params_insert_shape])
#print(params_insert_shape)

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)
torch.gather(params.expand((1, 2)), 1, indices.reshape((1, 2)))

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1, 2])
indices.shape = torch.Size([1, 2])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch:  tensor([[0, 0]])
FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1, 2])
indices.shape = torch.Size([1, 2])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch_to_tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)

tf.Tensor([[ True  True]], shape=(1, 2), dtype=bool)


tensor([[0, 0]])

In [ ]:
params = torch.tensor([0])

indices = torch.tensor([0, 0])

paramstf = tf.constant([0])

indicestf = tf.constant([0, 0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0 0], shape=(2,), dtype=int32)


FINAL SHAPE: torch.Size([2]), indices.shape[batch_dims:] = torch.Size([2])
indices.shape = torch.Size([2])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch:  tensor([0, 0])
FINAL SHAPE: torch.Size([2]), indices.shape[batch_dims:] = torch.Size([2])
indices.shape = torch.Size([2])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch_to_tf:  tf.Tensor([0 0], shape=(2,), dtype=int32)

tf.Tensor([ True  True], shape=(2,), dtype=bool)


In [ ]:
params = torch.tensor([-1])

indices = torch.tensor([[0]])

paramstf = tf.constant([-1])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([[-1]])
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([[-1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch:  tensor([[-1]])
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([0])

indices = torch.tensor([0])

paramstf = tf.constant([0])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0], shape=(1,), dtype=int32)


FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([0])
FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_to_tf:  tf.Tensor([0], shape=(1,), dtype=int32)

tf.Tensor([ True], shape=(1,), dtype=bool)


In [ ]:
params = torch.tensor([[0],
                 [0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0],
                 [0]])

indicestf = tf.constant([0])

axis = 1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[0]
 [0]], shape=(2, 1), dtype=int32)


FINAL SHAPE: torch.Size([2, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([2]), torch.Size([]), 2
INDICES SHAPE: torch.Size([2, -1]), torch.Size([2, 1])
torch:  tensor([[0],
        [0]])
FINAL SHAPE: torch.Size([2, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([2]), torch.Size([]), 2
INDICES SHAPE: torch.Size([2, -1]), torch.Size([2, 1])
torch_to_tf:  tf.Tensor(
[[0]
 [0]], shape=(2, 1), dtype=int32)

tf.Tensor(
[[ True]
 [ True]], shape=(2, 1), dtype=bool)


In [ ]:
params = torch.tensor([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indices = torch.tensor([3, 4, 1])

paramstf = tf.constant([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indicestf = tf.constant([3, 4, 1])

axis = 1
batch_dims = 0
print((torch.tensor(indices.shape[batch_dims:]) != params.shape[axis]))
print(f"BATCH DIMS: {batch_dims}")
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([True])
BATCH DIMS: 0
tf:  tf.Tensor([[-2072 -7720 25013]], shape=(1, 3), dtype=int32)


FINAL SHAPE: torch.Size([1, 3]), indices.shape[batch_dims:] = torch.Size([3])
indices.shape = torch.Size([3])
params.shape = torch.Size([1, 7])
STACK_DIMS_1_2: torch.Size([1]), torch.Size([]), 2
INDICES SHAPE: torch.Size([1, -1]), torch.Size([1, 3])
torch:  tensor([[-2072, -7720, 25013]])
FINAL SHAPE: torch.Size([1, 3]), indices.shape[batch_dims:] = torch.Size([3])
indices.shape = torch.Size([3])
params.shape = torch.Size([1, 7])
STACK_DIMS_1_2: torch.Size([1]), torch.Size([]), 2
INDICES SHAPE: torch.Size([1, -1]), torch.Size([1, 3])
torch_to_tf:  tf.Tensor([[-2072 -7720 25013]], shape=(1, 3), dtype=int32)

tf.Tensor([[ True  True  True]], shape=(1, 3), dtype=bool)


In [ ]:
params = torch.tensor([[-1, -1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1, -1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0



print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1 -1]], shape=(1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch:  tensor([[-1, -1]])
FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch_to_tf:  tf.Tensor([[-1 -1]], shape=(1, 2), dtype=int32)

tf.Tensor([[ True  True]], shape=(1, 2), dtype=bool)


In [ ]:
params = torch.tensor([[-1]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[[-1]]], shape=(1, 1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch:  tensor([[[-1]]])
FINAL SHAPE: torch.Size([1, 1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch_to_tf:  tf.Tensor([[[-1]]], shape=(1, 1, 1), dtype=int32)

tf.Tensor([[[ True]]], shape=(1, 1, 1), dtype=bool)


In [ ]:
a = torch.tensor([[1]])
print(a.reshape((1, 1, 1)))

In [ ]:
params = torch.tensor([[-1], [-1], [-1]])

indices = torch.tensor([1])

paramstf = tf.constant([[-1], [-1], [-1]])

indicestf = tf.constant([1])

axis = 0
batch_dims = 0
print(indices.unsqueeze(-1).unsqueeze(-1))
print(params.squeeze(-1))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[[1]]])
tensor([-1, -1, -1])
2 1
tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([3, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch:  tensor([[-1]])
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([3, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
torch.gather(torch.tensor([[-1], [-1], [-1]]), 0, torch.tensor([[1]]))

In [ ]:
a = torch.tensor([[1, 2]])
a.shape
a0 = a.expand((3, 3, 2))
a = a0.reshape((3, 3, 1, 2))
a = a.expand((3, 3, 2, 2))
ai0 = torch.ones((3, 2))
ai = ai0.reshape((1, 3, 2, 1))
ai = ai.expand((3, 3, 2, 2))

b = [1 for dim in list(a.shape)]
c = torch.Size(b)
d = torch.ones(c)

restorch = torch.gather(a, 1, ai.long())
torch_to_tf_a = tf.convert_to_tensor(a0, dtype = "int32")
torch_to_tf_ai = tf.convert_to_tensor(ai0, dtype = "int32")
print(torch_to_tf_a.shape, torch_to_tf_ai.shape)
restf = tf.gather(torch_to_tf_a, torch_to_tf_ai, axis = 1, batch_dims = 0)
print(restf)
print()
print(restorch.shape)
restf == tf.convert_to_tensor(restorch, dtype = "int32")


(3, 3, 2) (3, 2)
tf.Tensor(
[[[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]], shape=(3, 3, 2, 2), dtype=int32)

torch.Size([3, 3, 2, 2])


<tf.Tensor: shape=(3, 3, 2, 2), dtype=bool, numpy=
array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]])>

In [ ]:
params = torch.tensor([1])

indices = torch.tensor([[0], [0]])

paramstf = tf.constant([1])

indicestf = tf.constant([[0], [0]])

axis = -1
batch_dims = 0

print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(params.shape[:axis] , indices.shape[batch_dims:] , params.shape[axis + 1:])
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print(tf_tensor == tf.convert_to_tensor(torch_tensor, dtype = "int32"))
print("torch: ", torch_tensor)
print()

torch.Size([1])
tensor([[0],
        [0]])
torch.Size([]) torch.Size([2, 1]) torch.Size([1])
1 2
tf:  tf.Tensor(
[[1]
 [1]], shape=(2, 1), dtype=int32)


FINAL SHAPE: torch.Size([2, 1]), indices.shape[batch_dims:] = torch.Size([2, 1])
indices.shape = torch.Size([2, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch:  tensor([[1],
        [1]])
FINAL SHAPE: torch.Size([2, 1]), indices.shape[batch_dims:] = torch.Size([2, 1])
indices.shape = torch.Size([2, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
tf.Tensor(
[[ True]
 [ True]], shape=(2, 1), dtype=bool)
torch:  tensor([[1],
        [1]])



In [ ]:
a = torch.tensor([[[1]]])
b = torch.tensor([-1, -1, -1])
axis = 0
batch_dims = 0
print(a.shape, b.shape)
print(a.reshape((1, 1, 1, 1)))
print(a.shape[:axis], b.shape, a.shape[axis + 1:])

a = a.expand((a.shape[:axis] + b.shape + a.shape[axis + 1:]))
print(a)

In [ ]:
params = torch.tensor([0,
                 0])

indices = torch.tensor([[0]])

paramstf = tf.constant([0,
                 0])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([[0]])
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_to_tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
a = torch.tensor([0])
print(a.dim())
a = torch.tensor([[0]])
print(a.dim())

1
2


In [ ]:
params = torch.tensor([[0,
                 0]])

indices = torch.tensor([[0]])
print(torch.tensor([(torch.tensor(indices.shape[batch_dims:]) == params.shape[axis:axis + 1]) * params.shape[axis]]).long()
        + torch.tensor([torch.tensor(indices.shape[batch_dims:]) != params.shape[axis:axis + 1]]).long())
paramstf = tf.constant([[0,
                 0]])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([1])
tf:  tf.Tensor([[[0 0]]], shape=(1, 1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 1, 2]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch:  tensor([[[0, 0]]])
FINAL SHAPE: torch.Size([1, 1, 2]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch_to_tf:  tf.Tensor([[[0 0]]], shape=(1, 1, 2), dtype=int32)

tf.Tensor([[[ True  True]]], shape=(1, 1, 2), dtype=bool)


In [ ]:
params = torch.tensor([0, 0, 0])

indices = torch.tensor([0, 0, 0, 0])

paramstf = tf.constant([0, 0, 0])

indicestf = tf.constant([0, 0, 0, 0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0 0 0 0], shape=(4,), dtype=int32)


FINAL SHAPE: torch.Size([4]), indices.shape[batch_dims:] = torch.Size([4])
indices.shape = torch.Size([4])
params.shape = torch.Size([3])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([4])
torch:  tensor([0, 0, 0, 0])
FINAL SHAPE: torch.Size([4]), indices.shape[batch_dims:] = torch.Size([4])
indices.shape = torch.Size([4])
params.shape = torch.Size([3])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([4])
torch_to_tf:  tf.Tensor([0 0 0 0], shape=(4,), dtype=int32)

tf.Tensor([ True  True  True  True], shape=(4,), dtype=bool)


In [ ]:
params = torch.tensor([[0, 0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0, 0]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch:  tensor([[0, 0]])
FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1, 2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([2]), 2
INDICES SHAPE: torch.Size([-1, 2]), torch.Size([1, 2])
torch_to_tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)

tf.Tensor([[ True  True]], shape=(1, 2), dtype=bool)


In [ ]:
params = torch.tensor([0])

indices = torch.tensor([[0, 0]])

paramstf = tf.constant([0])

indicestf = tf.constant([[0, 0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1, 2])
indices.shape = torch.Size([1, 2])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch:  tensor([[0, 0]])
FINAL SHAPE: torch.Size([1, 2]), indices.shape[batch_dims:] = torch.Size([1, 2])
indices.shape = torch.Size([1, 2])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch_to_tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)

tf.Tensor([[ True  True]], shape=(1, 2), dtype=bool)


In [ ]:
a = torch.tensor([[0, 0, 0]])
a = a.expand(torch.Size([2, 3]))
print(a)
(torch.tensor(a.shape) == 2) * torch.tensor(a.shape) != torch.tensor([2, 0])
torch.tensor(a.shape) == 2
print(a.type())
a = a.to(dtype = torch.int8)
a.dtype

tensor([[0, 0, 0],
        [0, 0, 0]])
torch.LongTensor


torch.int8

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


AXIS: 0
torch:  []
AXIS: 0
torch_to_tf:  tf.Tensor([], shape=(0,), dtype=int32)

tf.Tensor(False, shape=(), dtype=bool)


tensor([[0, 1]])

In [ ]:
params = torch.tensor([0, 0])

indices = torch.tensor([[[0], [0]]])

paramstf = tf.constant([0, 0])

indicestf = tf.constant([[[0], [0]]])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[0]
  [0]]], shape=(1, 2, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 2, 1]), indices.shape[batch_dims:] = torch.Size([1, 2, 1])
indices.shape = torch.Size([1, 2, 1])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch:  tensor([[[0],
         [0]]])
FINAL SHAPE: torch.Size([1, 2, 1]), indices.shape[batch_dims:] = torch.Size([1, 2, 1])
indices.shape = torch.Size([1, 2, 1])
params.shape = torch.Size([2])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([2])
torch_to_tf:  tf.Tensor(
[[[0]
  [0]]], shape=(1, 2, 1), dtype=int32)

tf.Tensor(
[[[ True]
  [ True]]], shape=(1, 2, 1), dtype=bool)


In [ ]:

params = torch.tensor([[1],[0],[1]])

indices = torch.tensor([0])

paramstf = tf.constant([[1],[0],[1]])

indicestf = tf.constant([0])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[1]
 [0]
 [1]], shape=(3, 1), dtype=int32)


FINAL SHAPE: torch.Size([3, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([3, 1])
STACK_DIMS_1_2: torch.Size([3]), torch.Size([]), 2
INDICES SHAPE: torch.Size([3, -1]), torch.Size([3, 1])
torch:  tensor([[1],
        [0],
        [1]])
FINAL SHAPE: torch.Size([3, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([3, 1])
STACK_DIMS_1_2: torch.Size([3]), torch.Size([]), 2
INDICES SHAPE: torch.Size([3, -1]), torch.Size([3, 1])
torch_to_tf:  tf.Tensor(
[[1]
 [0]
 [1]], shape=(3, 1), dtype=int32)

tf.Tensor(
[[ True]
 [ True]
 [ True]], shape=(3, 1), dtype=bool)


In [ ]:

params = torch.tensor([0], dtype = torch.uint8)

indices = torch.tensor([[0]], dtype = torch.int8)

paramstf = tf.constant([0])

indicestf = tf.constant([[0]])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print("torch_tensor.type()= ", torch_tensor.type())
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([[0]], dtype=torch.uint8)
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1, 1])
indices.shape = torch.Size([1, 1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_tensor.type()=  torch.ByteTensor
torch_to_tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([0], dtype = torch.uint8)

indices = torch.tensor([0], dtype = torch.int32)

paramstf = tf.constant([0])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print("torch_tensor.type()= ", torch_tensor.type())
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0], shape=(1,), dtype=int32)


FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([0], dtype=torch.uint8)
FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_tensor.type()=  torch.ByteTensor
torch_to_tf:  tf.Tensor([0], shape=(1,), dtype=int32)

tf.Tensor([ True], shape=(1,), dtype=bool)


In [ ]:
params = torch.tensor([[0], [0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0], [0]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch:  tensor([[0]])
FINAL SHAPE: torch.Size([1, 1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([2, 1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([1]), 2
INDICES SHAPE: torch.Size([-1, 1]), torch.Size([1, 1])
torch_to_tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([1])

indices = torch.tensor([0])

paramstf = tf.constant([1])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([1], shape=(1,), dtype=int32)


FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch:  tensor([1])
FINAL SHAPE: torch.Size([1]), indices.shape[batch_dims:] = torch.Size([1])
indices.shape = torch.Size([1])
params.shape = torch.Size([1])
STACK_DIMS_1_2: torch.Size([]), torch.Size([]), 1
INDICES SHAPE: torch.Size([-1]), torch.Size([1])
torch_to_tf:  tf.Tensor([1], shape=(1,), dtype=int32)

tf.Tensor([ True], shape=(1,), dtype=bool)


In [ ]:
torch.gather(torch.tensor([1]), 0, torch.tensor([0]))

tensor([1])

In [ ]:
import tensorflow as tf

In [ ]:
t = tf.constant([[1, 2, 3],
     [4, 5, 6]])
dim = 1
tf.shape(t)[:dim]
t.shape[:dim] + tf.TensorShape((1))
tf.shape(t)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 3], dtype=int32)>

In [ ]:
t = tf.constant([[1, 2, 3, 7, 8, 9],
     [4, 5, 6, 9, 8, 7]])
dim = -1
shape = (3, 2)
len(t)
print(len(t.shape), tf.shape(t), tf.size(t))
dim = abs(len(t.shape) + dim) if dim < 0 else dim
res_shape = t.shape[:dim] + tf.TensorShape(shape) + t.shape[dim + 1 :]
print(dim, t.shape, t.shape[:dim], t.shape[dim + 1:])
res = tf.reshape(t, res_shape)
print(res)

2 tf.Tensor([2 6], shape=(2,), dtype=int32) tf.Tensor(12, shape=(), dtype=int32)
1 (2, 6) (2,) ()
tf.Tensor(
[[[1 2]
  [3 7]
  [8 9]]

 [[4 5]
  [6 9]
  [8 7]]], shape=(2, 3, 2), dtype=int32)


In [ ]:
import numpy as np

In [ ]:
t = np.array([[1, 2, 3, 7, 8, 9], [4, 5, 6, 9, 8, 7]])
dim = -1
shape = (3, 2)
len(t)
print(len(t.shape), np.shape(t))
dim = abs(len(t.shape) + dim) if dim < 0 else dim
res_shape = t.shape[:dim] + shape + t.shape[dim + 1 :]
print(dim, t.shape, t.shape[:dim], t.shape[dim + 1:])
res = np.reshape(t, res_shape)
print(res)

2 (2, 6)
1 (2, 6) (2,) ()
[[[1 2]
  [3 7]
  [8 9]]

 [[4 5]
  [6 9]
  [8 7]]]
